In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from pymongo import MongoClient

In [9]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}


result = ''
for page in range(1, 5):
    r = requests.get(f'https://novosibirsk.hh.ru/search/vacancy?page={page}', headers=headers)
    result+=r.content.decode()

soup=bs(result, 'html.parser')
vacancies = soup.find_all('div', {'class': 'vacancy-serp-item-body'})


entries = []
for vacancy in vacancies:
    
    title = (vacancy.find("a").text)
    try:
        salary = (vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'}).contents)
        currency = salary[-1]
        if 'от' in salary[0]:
            salary_min = int(salary[2].replace('\u202f',''))
            salary_max = '-'
        else:
            salary_min, salary_max = map(int, salary[0].replace('\u202f','').split(' – '));
    except:
        salary_min = salary_max = currency =  ('-')
    link = (vacancy.find('a')['href'].split('?')[0])
    entry = [title, salary_min, salary_max, currency, link]
    entries.append(entry)


df = pd.DataFrame(entries, columns=['title', 'salary_min', 'salary_max', 'currency', 'link'])
df.head()

,title,salary_min,salary_max,currency,link
0,Бармен/бариста,35000,-,руб.,https://novosibirsk.hh.ru/vacancy/55308309
1,"Комплектовщик / Упаковщик, Москва",86000,118000,руб.,https://novosibirsk.hh.ru/vacancy/55281358
2,Помощник кладовщика,53000,55000,руб.,https://novosibirsk.hh.ru/vacancy/55285723
3,Продавец-консультант,800,1400,бел. руб.,https://novosibirsk.hh.ru/vacancy/55328132
4,Менеджер по продажам,350000,-,руб.,https://novosibirsk.hh.ru/vacancy/55292889


In [34]:
mongo = MongoClient('mongodb://localhost:27017')

db = mongo['gbru']
collection = db['headhunter']

def update_vacancies():
    for index, row in df.iterrows():
        cur_link = row['link']
        if not collection.find_one({'link':cur_link}):
            collection.insert_one(row.to_dict())
        else:
            collection.update_one({'link': cur_link}, {'$set': row.to_dict()})
        
update_vacancies()   

for entry in collection.find():
    print(entry)

{'_id': ObjectId('627cf7f69f380bfa2b12047d'), 'title': 'Бармен/бариста', 'salary_min': 35000, 'salary_max': '-', 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55308309'}
{'_id': ObjectId('627cf7f69f380bfa2b12047e'), 'title': 'Комплектовщик / Упаковщик, Москва', 'salary_min': 86000, 'salary_max': 118000, 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55281358'}
{'_id': ObjectId('627cf7f69f380bfa2b12047f'), 'title': 'Помощник кладовщика', 'salary_min': 53000, 'salary_max': 55000, 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55285723'}
{'_id': ObjectId('627cf7f69f380bfa2b120480'), 'title': 'Продавец-консультант', 'salary_min': 800, 'salary_max': 1400, 'currency': 'бел.\xa0руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55328132'}
{'_id': ObjectId('627cf7f69f380bfa2b120481'), 'title': 'Менеджер по продажам', 'salary_min': 350000, 'salary_max': '-', 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55292889'}
{'_id': ObjectId